In [0]:
from bs4 import BeautifulSoup
import requests
import os

In [0]:
def request_webpage(url):
  res = requests.get(url)
  try:
    res.raise_for_status()
  except Exception as exp:
    print("Error: ", exp)
  return res

In [0]:
coming_soon_page = request_webpage("https://www.imdb.com/movies-coming-soon/2019-01/")

In [53]:
coming_soon_page.text

'\n\n\n\n\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n         \n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">\n\n\n\n        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};</script>\n\n<script>\n    if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n</script>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>\n        <title>New Movies Coming Soon - IMDb</title>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>\n<script>\n    if (typeof uet == \'function\') {\n      uet("be", "LoadTitle", {wb: 1});\n    }\n</script>\n<script>\n    if (typeof uex =

In [0]:
coming_soon_soup = BeautifulSoup(coming_soon_page.text)
# print(coming_soon_soup.prettify())

In [55]:
details = coming_soon_soup.find('div', attrs={'class': 'list detail'})
image_details = details.find_all('img')

# print(image_details)

image_list = [img['src'] for img in image_details if 'poster' in img['class']]
print(image_list)

name_list = [img['alt'].replace(' Poster', '') for img in image_details if 'poster' in img['class']]
print(name_list)

movie_list = []

for i in range(len(name_list)):
  movie_list.append([name_list[i], image_list[i]])
  
from tabulate import tabulate
print(tabulate(movie_list, headers = ['name', 'url'], tablefmt = 'orgtbl'))

['https://m.media-amazon.com/images/M/MV5BMjQ2NDMwMTY3MF5BMl5BanBnXkFtZTgwNDg5OTc1NjM@._V1_UY209_CR0,0,140,209_AL_.jpg', 'https://m.media-amazon.com/images/M/MV5BZDFhMWIwMzktNGQ4NC00MzYxLTliMjctMTM1NTc2YzVhZTkxXkEyXkFqcGdeQXVyNjk2MjA4MDY@._V1_UY209_CR8,0,140,209_AL_.jpg', 'https://m.media-amazon.com/images/M/MV5BYTAzYzI1ZDYtNDUwYS00OTg4LTg4N2ItMTk5MTUxYzgyMGM4XkEyXkFqcGdeQXVyMTk2MDc1MjQ@._V1_UY209_CR3,0,140,209_AL_.jpg', 'https://m.media-amazon.com/images/M/MV5BODI5Y2I0MTktZDRkNS00NWFjLTg1MmEtNWFkMzEwOGM0ZDE3XkEyXkFqcGdeQXVyNTA1NjYyMDk@._V1_UY209_CR0,0,140,209_AL_.jpg', 'https://m.media-amazon.com/images/M/MV5BNzY3NzYyNjI0N15BMl5BanBnXkFtZTgwNjYzMDc0NjM@._V1_UY209_CR0,0,140,209_AL_.jpg', 'https://m.media-amazon.com/images/M/MV5BMTg5MjcwMzY5OV5BMl5BanBnXkFtZTgwMDM0OTI1NjM@._V1_UY209_CR0,0,140,209_AL_.jpg', 'https://m.media-amazon.com/images/M/MV5BMDMzMThjZDgtYjhmNy00M2M3LWJlMGQtMzA5NTlhNWNmMDU3XkEyXkFqcGdeQXVyMTA0MjU0Ng@@._V1_UY209_CR3,0,140,209_AL_.jpg', 'https://m.media-amazon.com/ima

In [0]:
def get_fullsize_image(url):
  image_url = url
  index = image_url.find("_V1_")
  image_url = image_url[:index] + "_V1_.jpg"
  return image_url

In [57]:
get_fullsize_image(image_list[0])

'https://m.media-amazon.com/images/M/MV5BMjQ2NDMwMTY3MF5BMl5BanBnXkFtZTgwNDg5OTc1NjM@._V1_.jpg'

In [62]:
current_date = '2019-01'

try:
  os.makedirs(current_date)
except Exception as exp:
  print("Error: ", 'failed gracefully, you probably already made the folder.')

Error:  failed gracefully, you probably already made the folder.


In [0]:
def download_month_of_posters(images, names, date):
  try:
    os.makedirs(date)
  except Exception as exp:
    print('Error: ', 'failed gracefully, you probably already made the folder.')
    
  for i in range(len(images)):
    image_url = images[i]
    
    full_size_image_url = get_fullsize_image(image_url)
    img_res = request_webpage(full_size_image_url)
    
    name = names[i]
    
    if ('/' in name): # because file names can't have a slash
      name = name.replace('/', '-') 
      
    try:
      imageFile = open(os.path.join(date, name + '.jpg'), 'wb')
      size = 0
      
      for chunk in img_res.iter_content(100000):
        size = size + len(chunk)
        imageFile.write(chunk)
        
      print(name, size/1000, "KB")
      imageFile.close()
    except Exception as exc:
      print('There was a problem with writing the file for %s: %s' % (names[i], exc))
    
  print('All Finished with %s' % (date))

In [66]:
download_month_of_posters(image_list, name_list, current_date)

Error:  failed gracefully, you probably already made the folder.
Escape Room (2019) 163.377 KB
Great Great Great (2017) 401.915 KB
Yun nan chong gu (2018) 778.68 KB
The Nun (1966) 1942.341 KB
The Upside (2017) 1348.387 KB
A Dog's Way Home (2019) 121.668 KB
Perfectos desconocidos (2017) 228.913 KB
Replicas (2018) 1207.06 KB
The Untold Story (2019) 11022.158 KB
SGT. Will Gardner (2019) 521.194 KB
The Aspern Papers (2018) 431.88 KB
Glass (2019) 286.814 KB
Las herederas (2018) 159.829 KB
An Acceptable Loss (2018) 1954.479 KB
Adult Life Skills (2016) 3566.38 KB
Serenity (2019) 4495.304 KB
The Kid Who Would Be King (2019) 1911.643 KB
Die Unsichtbaren (2017) 4756.726 KB
Werk ohne Autor (2018) 151.951 KB
The Final Wish (2018) 235.56 KB
In Like Flynn (2018) 2055.48 KB
Jihadists (2016) 511.604 KB
Bricked (2019) 4213.768 KB
All Finished with 2019-01
